In [1]:
import torch
from lora_adapters import LoraConv2d, LoraLinear, LoraEmbedding, apply_adapter, mark_only_lora_as_trainable, lora_state_dict, undo_lora
from torch.optim import AdamW
from transformers import AutoModel, AutoTokenizer, AutoModelForSequenceClassification
from datasets import load_dataset, concatenate_datasets
from transformers import TrainingArguments, Trainer, default_data_collator, BertModel
import numpy as np
import evaluate

In [2]:
model = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=5).to('cuda')
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [3]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [4]:
dataset = load_dataset("yelp_review_full")

Found cached dataset yelp_review_full (/home/kian/.cache/huggingface/datasets/yelp_review_full/yelp_review_full/1.0.0/e8e18e19d7be9e75642fc66b198abadb116f73599ec89a69ba5dd8d1e57ba0bf)


  0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
dataset

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 650000
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 50000
    })
})

In [6]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)
tokenized_datasets = dataset.map(tokenize_function, batched=True)

Loading cached processed dataset at /home/kian/.cache/huggingface/datasets/yelp_review_full/yelp_review_full/1.0.0/e8e18e19d7be9e75642fc66b198abadb116f73599ec89a69ba5dd8d1e57ba0bf/cache-fbf6b44b4d6c6409.arrow
Loading cached processed dataset at /home/kian/.cache/huggingface/datasets/yelp_review_full/yelp_review_full/1.0.0/e8e18e19d7be9e75642fc66b198abadb116f73599ec89a69ba5dd8d1e57ba0bf/cache-03d7d9247ef0e135.arrow


In [7]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(100))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(100))

Loading cached shuffled indices for dataset at /home/kian/.cache/huggingface/datasets/yelp_review_full/yelp_review_full/1.0.0/e8e18e19d7be9e75642fc66b198abadb116f73599ec89a69ba5dd8d1e57ba0bf/cache-415b98b021217f20.arrow
Loading cached shuffled indices for dataset at /home/kian/.cache/huggingface/datasets/yelp_review_full/yelp_review_full/1.0.0/e8e18e19d7be9e75642fc66b198abadb116f73599ec89a69ba5dd8d1e57ba0bf/cache-4ddc9fe71d320244.arrow


In [8]:
metric = evaluate.load("accuracy")


In [9]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [10]:
# trainer.train()

In [11]:
USE_LORA = True
if USE_LORA:
    model = apply_adapter(model, LoraEmbedding, rank=16, regex_pattern=".*word_embeddings")
    model = apply_adapter(model, LoraLinear, rank=16, regex_pattern=".*query")
    model = apply_adapter(model, LoraLinear, rank=16, regex_pattern=".*value")
    model = mark_only_lora_as_trainable(model, bias="lora_only")
    model

In [12]:
optimizer = AdamW((param for param in model.parameters() if param.requires_grad), lr=1e-3)

In [13]:
training_args = TrainingArguments(output_dir="test_trainer", per_device_train_batch_size=48, per_device_eval_batch_size=64, report_to="none", bf16=True)

In [14]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
    optimizers=(optimizer, None)
)

In [15]:
trainer.train()

  0%|          | 0/6 [00:00<?, ?it/s]

/home/kian/anaconda3/envs/torch-2/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


{'train_runtime': 4.2378, 'train_samples_per_second': 70.791, 'train_steps_per_second': 1.416, 'train_loss': 1.6462214787801106, 'epoch': 3.0}


TrainOutput(global_step=6, training_loss=1.6462214787801106, metrics={'train_runtime': 4.2378, 'train_samples_per_second': 70.791, 'train_steps_per_second': 1.416, 'train_loss': 1.6462214787801106, 'epoch': 3.0})

In [16]:
model_parameters = sum(p.numel() for p in model.parameters())    
model_grads = sum(p.grad.numel() for p in model.parameters() if p.requires_grad and p.grad)    
optimizer_states = sum([sum(elem.numel() for elem in  p.values()) for p in optimizer.state.values()])
print(model_parameters, model_grads, optimizer_states)

110576549 0 2217866


In [17]:
# 109486085 0 218972371

In [18]:
grad_dict = {name: param for name, param in model.named_parameters() if param.requires_grad}

In [19]:
grad_dict.keys()

dict_keys(['bert.embeddings.word_embeddings.lora_A', 'bert.embeddings.word_embeddings.lora_B', 'bert.encoder.layer.0.attention.self.query.bias', 'bert.encoder.layer.0.attention.self.query.lora_A', 'bert.encoder.layer.0.attention.self.query.lora_B', 'bert.encoder.layer.0.attention.self.value.bias', 'bert.encoder.layer.0.attention.self.value.lora_A', 'bert.encoder.layer.0.attention.self.value.lora_B', 'bert.encoder.layer.1.attention.self.query.bias', 'bert.encoder.layer.1.attention.self.query.lora_A', 'bert.encoder.layer.1.attention.self.query.lora_B', 'bert.encoder.layer.1.attention.self.value.bias', 'bert.encoder.layer.1.attention.self.value.lora_A', 'bert.encoder.layer.1.attention.self.value.lora_B', 'bert.encoder.layer.2.attention.self.query.bias', 'bert.encoder.layer.2.attention.self.query.lora_A', 'bert.encoder.layer.2.attention.self.query.lora_B', 'bert.encoder.layer.2.attention.self.value.bias', 'bert.encoder.layer.2.attention.self.value.lora_A', 'bert.encoder.layer.2.attention.s

In [20]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): LoraEmbedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): LoraLinear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): LoraLinear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,)

In [21]:
input_tensor = torch.randint(0, 512, (1,256)).to('cuda')

In [22]:
output = model(input_tensor)

In [23]:
output.logits

tensor([[0.0468, 0.0428, 0.0396, 0.2946, 0.0919]], device='cuda:0',
       grad_fn=<AddmmBackward0>)

In [24]:
model = AutoModel.from_pretrained('bert-base-uncased', num_labels=5).to('cuda')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [25]:
output = model(input_tensor)

In [28]:
output.last_hidden_state.sum()

tensor(-2523.9951, device='cuda:0', grad_fn=<SumBackward0>)